<a href="https://colab.research.google.com/github/minsaee/ai_chat_project01/blob/master/002_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ai_chat_project01'

In [ ]:
!python -V

In [ ]:
!pip install langchain openai==0.28.1 chromadb

In [ ]:
!pip install tiktoken

In [ ]:
import os

os.environ["OPENAI_API_KEY"]='sk-'

In [ ]:
import openai
import chromadb
import langchain

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains import ChatVectorDBChain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from openai.embeddings_utils import get_embedding

In [ ]:
!pwd

/content/drive/MyDrive/ai_chat_project01


In [ ]:
# data 가져오기

loader = DirectoryLoader('./m_data/data/flour', loader_cls=TextLoader)
data = loader.load()

print(f"{len(data)}개의 문서를 가지고 있습니다.")
print(f"문서에{len(data[0].page_content)}개의 단어를 가지고 있습니다.")

392개의 문서를 가지고 있습니다.
문서에875개의 단어를 가지고 있습니다.


In [ ]:
# text 분할하기

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(data)

print('분할된 텍스트의 개수:', len(texts))

분할된 텍스트의 개수: 3387


In [ ]:
embedding = OpenAIEmbeddings()
persist_directory = './DB/flour'
vectordb = Chroma.from_documents(
    documents = texts,
    embedding = embedding,
    persist_directory=persist_directory
)

vectordb

In [ ]:
embedding = OpenAIEmbeddings()
db = Chroma( persist_directory='./DB',  embedding_function=embedding)

In [ ]:
retriever = db.as_retriever()

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0),
    chain_type = 'stuff',
    retriever = retriever,
    return_source_documents = True)

In [ ]:
def get_chatbot_response(chatbot_response):
  # print(chatbot_response['result'].strip())
  print(chatbot_response['result'])
  print('\n문서 출처:')
  for source in chatbot_response['source_documents']:
    print(source.metadata['source'])

In [ ]:
def get_chatbot_response(chatbot_response):
  # print(chatbot_response['result'].strip())
  print(chatbot_response['result'])
  print('\n문서 출처:')
  for source in chatbot_response['source_documents']:
    print(source.metadata['source'])
    with open(f"./{source.metadata['source']}", "r") as f:
      text = f.read()
      print(text)

In [ ]:
### 요약 + 요리제목 5가지
import re

def get_chatbot_response(chatbot_response):
  print(chatbot_response['result'].strip())
  print('\n요리 상세정보:')
  for source in chatbot_response['source_documents']:
    s = source.metadata['source']
    numbers = re.findall(r'\d+', s)
    n = (''.join(numbers))
    print(f'https://www.10000recipe.com/recipe/{n}')
  print('\n추천하는 요리:')
  for i, source in enumerate(chatbot_response['source_documents']):
    print(source.metadata['source'])
    with open(f"./{source.metadata['source']}", "r") as f:
      text = f.readlines()[0]
      print('{}.{}'.format(i+1, text.strip()))

In [ ]:
### 요약 + 요리제목 5가지
import re

def get_chatbot_response(chatbot_response):
  print(chatbot_response['result'].strip())
  print('\n추천하는 요리:')
  for i, source in enumerate(chatbot_response['source_documents']):
    s = source.metadata['source']
    numbers = re.findall(r'\d+', s)
    n = (''.join(numbers))
    # print(source.metadata['source'])
    print(source.metadata)
    with open(f"./{source.metadata['source']}", "r") as f:
      text = f.readlines()[0]
      print('{}.{}'.format(i+1, text.strip()), f'https://www.10000recipe.com/recipe/{n}')

In [ ]:
input_text = '다시마 쌈 레시피 알려줘.'
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

In [ ]:
def get_chatbot_response(chatbot_response):
    response_text = chatbot_response['result'].strip() + '\n\n추천하는 요리:\n'

    for i, source in enumerate(chatbot_response['source_documents']):
        s = source.metadata['source']
        numbers = re.findall(r'\d+', s)
        n = (''.join(numbers))

        with open(f"./{source.metadata['source']}", "r") as f:
            text = f.readlines()[0]
            response_text += '{}.{} {}\n'.format(i+1, text.strip(), f'https://www.10000recipe.com/recipe/{n}')

    return response_text

In [ ]:
get_chatbot_response(chatbot_response)

쌈다시마와 양파를 함께 사용하는 방법에 대해 알려드릴게요.

1. 대파를 잘게 다지고 양파도 작게 다져주세요.
2. 팬에 식용유를 두르고 중불에서 양파를 볶아주세요.
3. 양파가 투명해질 때까지 볶은 후 쌈다시마를 넣고 함께 볶아주세요.
4. 쌈다시마가 익을 때까지 중간 불에서 볶아주세요.
5. 볶은 쌈다시마와 양파를 다른 요리에 사용하거나, 밥이나 국수 위에 올려서 드셔도 좋아요.

이렇게 쌈다시마와 양파를 함께 사용하여 맛있는 요리를 만들어보세요!

추천하는 요리:
1.﻿단짠 단짠 간장 돼지 불고기 만들기~ https://www.10000recipe.com/recipe/7009621
2.﻿명란 어묵전~* https://www.10000recipe.com/recipe/6926421
3.﻿ 별모양 두부참치전 https://www.10000recipe.com/recipe/3660582
4.﻿삼겹깐풍덮밥 : 비 오는 날에 생각나는 삼겹살!!!! 실패없는 깐풍소스로 덮밥만들기 ♥ https://www.10000recipe.com/recipe/6935331


In [ ]:
print(type(
get_chatbot_response(chatbot_response)))

<class 'str'>


In [ ]:
input_text = '계란후라이의 자세한 재료와 조리순서 알려줘.'

In [ ]:
chatbot_response = qa_chain(input_text)

In [ ]:
get_chatbot_response(chatbot_response)

재료:
- 계란
- 기름

조리순서:
1. 계란을 풀어서 그릇에 넣습니다.
2. 다진 김밥 재료를 계란에 넣고 소금을 약간 넣어줍니다.
3. 팬에 기름을 두르고 계란을 올려줍니다.
4. 계란을 돌돌 말아 얇게 썰어줍니다.
5. 계란후라이가 완성되면 다른 요리에 사용하거나 볶음밥 위에 얹어줍니다.

추천하는 요리:
1.﻿고추장 햄볶음밥 https://www.10000recipe.com/recipe/6886947
2.﻿예쁜김밥 세가지 한번 해봐요~~^^ https://www.10000recipe.com/recipe/6876144
3.﻿볶음밥 https://www.10000recipe.com/recipe/6828161
4.﻿폭신폭신 계란지단 잔뜩 넣은 '경주교리김밥'레시피 https://www.10000recipe.com/recipe/6895795


In [ ]:
a = get_chatbot_response(chatbot_response)

재료:
- 계란
- 기름

조리순서:
1. 계란을 풀어서 그릇에 넣습니다.
2. 다진 김밥 재료를 계란에 넣고 소금을 약간 넣어줍니다.
3. 팬에 기름을 두르고 계란을 올려줍니다.
4. 계란을 돌돌 말아 얇게 썰어줍니다.
5. 계란후라이가 완성되면 다른 요리에 사용하거나 볶음밥 위에 얹어줍니다.

추천하는 요리:
1.﻿고추장 햄볶음밥 https://www.10000recipe.com/recipe/6886947
2.﻿예쁜김밥 세가지 한번 해봐요~~^^ https://www.10000recipe.com/recipe/6876144
3.﻿볶음밥 https://www.10000recipe.com/recipe/6828161
4.﻿폭신폭신 계란지단 잔뜩 넣은 '경주교리김밥'레시피 https://www.10000recipe.com/recipe/6895795


In [ ]:
input_text = '계란후라이의 자세한 재료와 조리순서 알려줘.'
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

재료:
- 계란
- 기름

조리순서:
1. 계란을 풀어서 그릇에 넣고 잘 섞어줍니다.
2. 팬에 기름을 두르고 중약불로 예열해줍니다.
3. 예열된 팬에 계란을 부어서 얇게 펴줍니다.
4. 계란이 익을 때까지 한쪽 면을 구워줍니다.
5. 구워진 면을 접어서 반숙 후라이 형태로 만들어줍니다.
6. 반숙 후라이를 팬에서 꺼내어서 썰어줍니다.

이렇게 계란 후라이가 완성됩니다.

문서 출처:
m_data/data/rice/6886947.txt
m_data/data/rice/6876144.txt
m_data/data/rice/6828161.txt
m_data/data/rice/6895795.txt


In [ ]:
input_text = '계란후라이의 자세한 재료와 조리순서 알려줘.'
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

재료:
- 계란
- 기름

조리순서:
1. 계란을 풀어서 그릇에 넣습니다.
2. 다진 김밥 재료를 계란에 넣고 소금을 약간 넣어줍니다.
3. 팬에 기름을 두르고 계란을 올려줍니다.
4. 계란을 돌돌 말아 얇게 썰어줍니다.
5. 계란후라이가 완성되면 다른 요리에 사용하거나 볶음밥 위에 얹어줍니다.

추천하는 요리:
{'source': 'm_data/data/rice/6886947.txt'}
1.﻿고추장 햄볶음밥 https://www.10000recipe.com/recipe/6886947
{'source': 'm_data/data/rice/6876144.txt'}
2.﻿예쁜김밥 세가지 한번 해봐요~~^^ https://www.10000recipe.com/recipe/6876144
{'source': 'm_data/data/rice/6828161.txt'}
3.﻿볶음밥 https://www.10000recipe.com/recipe/6828161
{'source': 'm_data/data/rice/6895795.txt'}
4.﻿폭신폭신 계란지단 잔뜩 넣은 '경주교리김밥'레시피 https://www.10000recipe.com/recipe/6895795
